In [1]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

tcmalloc: large alloc 1073750016 bytes == 0x5cad6000 @  0x7f3ef23871c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
0.4.0
True


In [2]:
!pip3 install Pillow==4.0.0
!pip3 install PIL
!pip3 install image

    100% |████████████████████████████████| 5.6MB 5.4MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.2.0
    Uninstalling Pillow-5.2.0:
      Successfully uninstalled Pillow-5.2.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL
    100% |████████████████████████████████| 7.1MB 4.0MB/s 


In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets
import torch.nn as nn
import torch.nn.functional as F

In [0]:
trans = transforms.Compose([transforms.ToTensor()])

In [5]:
trainset = torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=trans)

Processing...
Done!


In [0]:
testset = torchvision.datasets.MNIST(root='./data',train=False,download=True,transform=trans)

In [0]:
train_loader = torch.utils.data.DataLoader(
                 dataset=trainset,
                 shuffle=True)

In [0]:
test_loader = torch.utils.data.DataLoader(dataset=testset,shuffle=True)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32,5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64,64,3,padding=1)
        self.conv4 = nn.Conv2d(64,64,3,padding=1)
        self.fc1 = nn.Linear(1024, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = F.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        residual = x
        x = (F.relu(self.conv3(x)))
        x = (F.relu(self.conv4(x)+residual))
        x = x.view(-1, 1024)
        x = F.relu(self.fc1(x))
        return x

In [0]:
import torch.optim as optim

In [0]:
loss_func = nn.CrossEntropyLoss()

In [0]:
model = Net()

In [0]:
model = model.cuda()

In [0]:
optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.95)

In [0]:
from torch.autograd import Variable

In [16]:
for epoch in range(4):
  for i,(x,y) in enumerate(train_loader):
    
    x = x.cuda()
    y=y.cuda()
    inputs, labels = Variable(x),Variable(y)
    optimizer.zero_grad()
    answer = model(inputs)
    loss = loss_func(answer,labels)
    loss.backward()
    optimizer.step()
    if(i%10000==0):
      print(loss)

tensor(2.3351, device='cuda:0')
tensor(2.3026, device='cuda:0')
tensor(2.3026, device='cuda:0')
tensor(2.3026, device='cuda:0')
tensor(2.3026, device='cuda:0')
tensor(1.00000e-04 *
       1.0586, device='cuda:0')
tensor(1.00000e-03 *
       3.3908, device='cuda:0')
tensor(0.1099, device='cuda:0')
tensor(1.00000e-04 *
       1.8787, device='cuda:0')
tensor(1.3855, device='cuda:0')
tensor(1.00000e-05 *
       2.5749, device='cuda:0')
tensor(0., device='cuda:0')
tensor(1.00000e-02 *
       1.6644, device='cuda:0')
tensor(1.00000e-04 *
       3.8910, device='cuda:0')
tensor(1.00000e-04 *
       3.4904, device='cuda:0')
tensor(1.00000e-02 *
       1.0379, device='cuda:0')
tensor(1.00000e-04 *
       2.2602, device='cuda:0')
tensor(1.00000e-05 *
       3.0518, device='cuda:0')
tensor(0., device='cuda:0')
tensor(1.00000e-06 *
       1.9073, device='cuda:0')
tensor(1.00000e-06 *
       7.6294, device='cuda:0')
tensor(1.00000e-04 *
       9.6321, device='cuda:0')
tensor(1.00000e-06 *
       3.8

In [0]:
correct=0
total=len(test_loader)

In [18]:
total

10000

In [0]:
with torch.no_grad():
    for image,label in test_loader:
        image = image.cuda()
        label = label.cuda()
        output = model(image)
        pred = output.max(1,True)[1]
        if(pred==label):
          correct+=1

In [20]:
correct



9903

In [21]:
print('Accuracy:'+str(correct/total))

Accuracy:0.9903


In [0]:
correct=0

In [0]:
with torch.no_grad():
    for image,label in train_loader:
        image = image.cuda()
        label = label.cuda()
        output = model(image)
        pred = output.max(1,True)[1]
        if(pred==label):
          correct+=1

In [0]:
total = len(train_loader)

In [25]:
print('Training set accuracy: '+str(correct/total))

Training set accuracy: 0.9949666666666667
